# LangChain Part 3: Output Parsing
- Important when using agents with tools e.g. calculator , search engine 
- Many systems expect data in specific way

In [37]:
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama
from os import getenv
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model="meta-llama/llama-3.2-3b-instruct:free",
    temperature=0.2,
)
# llm = ChatOllama(model='llama3.2', temperature=0.2, max_tokens=512)

result = llm.invoke("Hello, how are you today?")
print(result.content)

I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you have. How can I assist you today?


# CSV Parser 
- Used when want to return a list of comma-separated Items

In [38]:
from functools import partial
from pprint import pp
from tempfile import template
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate


output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print("Format Instructions:\n",format_instructions, "\n")

prompt = PromptTemplate(
    template="List five {subject}. \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

pprint(prompt.dict())

# Create a model that uses the prompt
chain = prompt | llm | output_parser

# Invoke the model
result = chain.invoke({"subject": "dog breeds"})

print('\nResult:\n',result)




Format Instructions:
 Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz` 

{'_type': 'prompt',
 'input_variables': ['subject'],
 'metadata': None,
 'name': None,
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'format_instructions': 'Your response should be a list '
                                              'of comma separated values, eg: '
                                              '`foo, bar, baz` or '
                                              '`foo,bar,baz`'},
 'tags': None,
 'template': 'List five {subject}. \n{format_instructions}',
 'template_format': 'f-string',
 'validate_template': False}

Result:
 ['Labrador', 'German Shepherd', 'Poodle', 'Beagle', 'Rottweiler']


# Datetime Parser
- Used when want to return a date in a specific format
- Useful for applications that require a specific date format like a calendar

In [41]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate

# Create a new prompt template for datetime parsing
datetime_prompt = PromptTemplate(
    template="What is the current date and time in UTC?\n{format_instructions}",
    input_variables=["format_instructions"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# Create a chain with the new prompt and the existing llm and output_parser
datetime_chain = datetime_prompt | llm | output_parser

# Invoke the chain
datetime_result = datetime_chain.invoke({})

print('\nDatetime Result:\n', datetime_result)



Datetime Result:
 2024-11-20 14:41:04.123456


# Enum Parser
- Used when want to return a list of items in a specific format
- e.g. list of colors, list of countries
- Useful for applications that require a specific list of items like a dropdown menu.

In [46]:
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from enum import Enum

# Define an enum for the days of the week
class DayOfWeek(Enum):
    SUNDAY = "Sunday"
    MONDAY = "Monday"
    TUESDAY = "Tuesday"
    WEDNESDAY = "Wednesday"
    THURSDAY = "Thursday"
    FRIDAY = "Friday"
    SATURDAY = "Saturday"
    
# Create a new output parser for the enum
enum_output_parser = EnumOutputParser(enum_class=DayOfWeek)

# Create a new prompt template for the enum
enum_prompt = PromptTemplate(
    template="What day of the week is it today?\n{format_instructions}",
    input_variables=["format_instructions"],
    partial_variables={"format_instructions": enum_output_parser.get_format_instructions()},
)

# Create a chain with the new prompt and the existing llm and output_parser
enum_chain = enum_prompt | llm | enum_output_parser

# Invoke the chain
enum_result = enum_chain.invoke({})

print('\nEnum Result:\n', enum_result)

KeyError: 'enum'


Datetime Result:
 2024-11-20 14:41:47.123456
